In [2]:
import time
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import random
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json 
from jupyter_ui_poll import ui_events

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

def countdown(timer):
    """
    Countdown function to display information about the test and a timer till the questions 
    are displayed
    Arguments: timer(the duration of the countdown in seconds)
    Returns: none
    """
    while timer:
        intro_to_test = f"""Welcome to the Maths Ability Test <br>
        This test will begin by asking questions about yourself <br>
        Then you will be asked maths questions <br> 
        This test starts in: {timer} seconds"""
        display(HTML(f"<h1>{intro_to_test}</h1>"))
        time.sleep(1)
        timer -= 1
        clear_output(wait=True)

    clear_output(wait=True)

timer = 8
countdown(timer)

id_instructions = """
Enter your anomised ID <br>
To generate an anonymous 4-letter unique user identifier please enter: <br>
Two letters based on the initials of a chldhood fiend <br> Two initials based on a favourite actor/ actress <br>
e.g. if your friend was called Charlie Brown and film star was Tom Cruise, then your unique identifier would be CBTC <br>
This should be the same as what is used for other tests done for group 6"""
display(HTML(f"<h1>{id_instructions}</h1>"))
user_id = input("> ")
clear_output(wait=True)

display(HTML("<h1>What is your age?</h1>"))
ans1 = input("> ")
clear_output(wait=True)

display(HTML("<h1>What is your gender?</h1>"))
ans2 = input("> ")
clear_output(wait=True)

display(HTML("<h1>Have you consumed any substances (Eg. caffeine, alcohol, or drugs) in the last 12 hours that might affect your cognitive abilities?</h1>"))
display(HTML("<h1>Note: Please enter 'yes' or 'no'. If yes please specify. Press enter if you would prefer not to disclose this information.</h1>"))
ans3 = input("> ")
clear_output(wait=True)



def countdown(timer):
    """
    Countdown function to display information about the test and a timer till the questions 
    are displayed
    Arguments: timer(the duration of the countdown in seconds)
    Returns: none
    """
    while timer:
        intro_2 = f"""This part of the test will ask you maths questions <br>
        Each question has 3 attempts <br>
        Each correct answer gains 2 points, each incorrect answer loses a point <br>
        Please enter whole numbers only <br>
        This test starts in: {timer} seconds"""
        display(HTML(f"<h1>{intro_2}</h1>"))
        # display(HTML("<h1>This part of the test will ask you maths questions</h1>"))
        # display(HTML("<h1>Each question has 3 attempts</h1>"))
        # display(HTML("<h1>Each correct answer gains 2 point, each incorrect answer looses a point</h1>"))
        # display(HTML("<h1>Please enter whole numbers only</h1>"))
        # display(HTML(f"<h1>This test starts in: {timer} seconds</h1>"))
        time.sleep(1)
        timer -= 1
        clear_output(wait=True)

    clear_output(wait=True)

timer = 8
countdown(timer)

clear_output(wait=True)

# The numbers that I use in my questions, the numbers get harder through each section as the questions progress
numbers_list = ['12', '13', '49', '36', '127']
numbers_list2 = ['11', '54', '65', '87', '52']
numbers_list3 = ['5', '11', '9', '14', '23']
numbers_list4 = ['7', '4', '12', '7', '11']
numbers_list5 = ['6', '12', '13', '34', '7']
numbers_list6 = ['18', '45', '56', '87', '135']
numbers_list7 = ['6', '9', '7', '3', '5']
numbers_list8 = ['33', '46', '52', '16', '46']

def addition_questions(number_one, number_two):
    """
    This asks the user 5 addition questions and allows the user to input the answer
    Arguments: number_one (the first number in the question, number_two (the second number
    in the question)
    Returns: a tuple containing time_taken (the time taken to answer each question, counter_add 
    (the score of the user) and incorrect_answers_add (the number of incorrect answers)
    """

    # To define the inital values as 0
    counter_add = 0
    incorrect_answers_add = 0

    # The formula for how the coreect answer is calculated
    correct_answer = int(number_one) + int(number_two)

    # Loop so each question has 3 attempts. If the user gets a question wrong, they loose an attempt
    # This loop will run till the user answers correctly or runs out of attempts
    attempts = 3
    while attempts >= 0:
        # These display the questions. Each number and operator is displayed for a short period of time then disappears
        display(HTML(f"<span style='font-size: 90px;'>{number_one}</span>"))
        time.sleep(2)
        clear_output()

        display(HTML("<span style='font-size: 90px;'>+</span>"))
        time.sleep(1)
        clear_output(wait=True)

        display(HTML(f"<span style='font-size: 90px;'>{number_two}</span>"))

        # Starts the timer
        start_time=time.time()

        #correct_answer = int(number_one) + int(number_two)

        # Allows the user to input their answer
        display(HTML("<h1>Enter the answer: </h1>"))
        user_answer = int(input("> "))
        if user_answer == correct_answer: # If the answer is correct this code runs
            display(HTML("<h1>Well done!</h1>"))
            time. sleep(1)
            counter_add += 2 # The user gains two points when correct
            break # If the user answers correctly the loop is broken 
        elif attempts>1: # This code runs if the user is incorrect but still has attempts left
            attempts -= 1 # Lowers the number of attempts
            counter_add -=1 # Lowers the number of attempts
            incorrect_answers_add += 1 # Increases the number incorrect
            display(HTML("<h1>Incorrect answer!</h1>"))
            display(HTML(f"<h1>You have {attempts} attempts left.</h1>"))
            time.sleep(2)
            clear_output(wait=True)
        else: # If the user runs out of attempts, the correct answer is displayed
            display(HTML(f"<h1>Incorrect answer! The correct answer was: {correct_answer}</h1>"))
            counter_add -=1 # Lowers the number of attempts
            incorrect_answers_add += 1 # Increases the number incorrect
            time.sleep(2)
            clear_output(wait=True)
            break

    # Ends the timer
    end_time = time.time()

    #Calculates the time taken per question
    time_taken = end_time - start_time

    return time_taken, counter_add, incorrect_answers_add

def mixed_questions(number_three, number_four, number_five):
    """
    This asks the user 5 questions involving multiplication and addition and allows the user to input the answer
    Arguments: number_three (the third number in the question), number_four (the fourth number
    in the question), number_five (the fifth number if the question)
    Returns: a tuple containing time_taken2 (the time taken to answer each question, counter 
    (the score of the user) and incorrect_answers_add (the number of incorrect answers)
    """

    # Defines the initial values as 0
    counter_mixed = 0
    incorrect_answers_mixed = 0

    # Defines how the correct answer for these questions is calculated
    correct_answer = int(number_three) * int(number_four) +int(number_five)

    attempts = 3 # Gives each question 3 attempts
    while attempts >= 0: # So the loop only runs when there are greater than or equal to zero attempts
        # Displays each number and operator for short periods of time before being cleared
        display(HTML(f"<span style='font-size: 90px;'>{number_three}</span>"))
        time.sleep(2)
        clear_output()

        display(HTML("<span style='font-size: 90px;'>x</span>"))
        time.sleep(1)
        clear_output(wait=True)

        display(HTML(f"<span style='font-size: 90px;'>{number_four}</span>"))
        time.sleep(2)
        clear_output(wait=True)

        display(HTML("<span style='font-size: 90px;'>+</span>"))
        time.sleep(1)
        clear_output(wait=True)

        display(HTML(f"<span style='font-size: 90px;'>{number_five}</span>"))

        # Starts the timer
        start_time=time.time()

        display(HTML("<h1>Enter the answer: </h1>"))
        user_answer = int(input("> ")) # To allow the user to enter their answer
        if user_answer == correct_answer: # If the user is correct, this code runs
            display(HTML("<h1>Well done!</h1>"))
            time.sleep(1)
            counter_mixed += 2 # User gains points if they answer correctly, then the loop breakks 
            break
        elif attempts>1: # If the loser get the questions wrong but still has attempters left
            attempts -= 1 # Looses an attempt
            counter_mixed -=1 # Looses a point
            incorrect_answers_mixed += 1 # Adds to the score of incorrect answers
            display(HTML("<h1>Incorrect answer!</h1>"))
            display(HTML(f"<h1>You have {attempts} attempts left.</h1>"))
            time.sleep(2)
            clear_output(wait=True)
        else: # If the user runs out of attempts the correct answer is displayed
            display(HTML(f"<h1>Incorrect answer! \nThe correct answer was: {correct_answer}</h1>"))
            counter_mixed -=1 # Looses a point
            incorrect_answers_mixed += 1 # Adds to the score of incorrect answers
            time.sleep(2)
            clear_output(wait=True)
            break

    # Stops the clock running
    end_time = time.time()

    # Calculates the time taken per question
    time_taken2 = end_time - start_time

    return time_taken2, counter_mixed, incorrect_answers_mixed

def mixed_questions2(number_six, number_seven, number_eight):
    """
    This asks the user 5 questions involving division and addition and allows the user to input the answer
    Arguments: number_six (the sixth number in the question), number_seven (the seventh number
    in the question), number_eigth (the eigth number if the question)
    Returns: a tuple containing time_taken3 (the time taken to answer each question, counter 
    (the score of the user) and incorrect_answers_add (the number of incorrect answers)
    """

    # Defines the initial values as 0
    counter_mixed2 = 0
    incorrect_answers_mixed2 = 0

    # Defines how these questions are answered
    correct_answer = int(number_six) / int(number_seven) + int(number_eight)

    # Loop to display the questions and allow the user to answer when they have attempts left
    attempts = 3
    while attempts >= 0:
        # Displays the numbers and operators used in these questions 
        display(HTML(f"<span style='font-size: 90px;'>{number_six}</span>"))
        time.sleep(2)
        clear_output()

        display(HTML("<span style='font-size: 90px;'>&divide;</span>"))
        time.sleep(1)
        clear_output(wait=True)

        display(HTML(f"<span style='font-size: 90px;'>{number_seven}</span>"))
        time.sleep(2)
        clear_output(wait=True)

        display(HTML("<span style='font-size: 90px;'>+</span>"))
        time.sleep(1)
        clear_output(wait=True)

        display(HTML(f"<span style='font-size: 90px;'>{number_eight}</span>"))

        # Starts the timer
        start_time=time.time()

        display(HTML("<h1>Enter the answer: </h1>"))
        user_answer = int(input("> ")) # To allow the user to answer
        if user_answer == correct_answer: # If the answer is correct, this code runs
            display(HTML("<h1>Well done!</h1>"))
            time. sleep(1)
            counter_mixed2 += 2 # The score increases when correct answer is given
            break
        elif attempts>1: # If the answer is wrong, this code runs
            attempts -= 1 # Lowers the numbers of attempts
            counter_mixed2 -=1 # Lowers the score
            incorrect_answers_mixed2 += 1 # Increases the number of incorrect answers
            display(HTML("<h1>Incorrect answer!</h1>"))
            display(HTML(f"<h1>You have {attempts} attempts left.</h1>"))
            time.sleep(2)
            clear_output(wait=True)
        else: # If the answer is wrong and the user has run out of attempts
            display(HTML(f"<h1>Incorrect answer! The correct answer was: {correct_answer}</h1>"))
            counter_mixed2 -=1 # Lowers the score
            incorrect_answers_mixed2 += 1 # Increases the number of incorrect attempts
            time.sleep(2)
            clear_output(wait=True)
            break

    # Stops the timer
    end_time = time.time()

    # Calculates the time taken per questions
    time_taken3 = end_time - start_time

    return time_taken3, counter_mixed2, incorrect_answers_mixed2 

# Puts all the types of questions into a list, so they can be shuffled
question_functions = [addition_questions, mixed_questions, mixed_questions2]

# Shuffle the list to randomize the order of question types
random.seed(1)  
random.shuffle(question_functions)

total_time = 0
total_time2 = 0
total_time3 = 0
total_counter_add = 0
total_counter_mixed = 0
total_counter_mixed2 = 0
total_incorrect_answers = 0
counter = 0

# Loop to go through the different types of questions in a random order
for func in question_functions:
    clear_output(wait=True)
    if func == addition_questions: # This code is run when addition questions are called on
        for num_one, num_two in zip(numbers_list, numbers_list2): # Takes numbers from numbers list one and numbers list 2
            clear_output(wait=True)
            time_taken, counter_add, incorrect_answers_add = addition_questions(num_one, num_two)
            total_time += time_taken # Adds together the time for each question in this question
            total_counter_add += counter_add # Adds to the score for addition questions
            total_incorrect_answers += incorrect_answers_add # Adds to total number of incorrect numbers
        addition_question_avg = total_time / len(numbers_list) # Calculates the average time per question
    elif func == mixed_questions: # This code is run when mixed questions are called on
        for num_three, num_four, num_five in zip(numbers_list3, numbers_list4, numbers_list5): # Uses numbers from list 3, 4, 5
            clear_output(wait=True)
            time_taken2, counter_mixed, incorrect_answers_mixed = mixed_questions(num_three, num_four, num_five)
            total_time2 += time_taken2 # Adds together the time for each question in this question
            total_counter_mixed += counter_mixed # Adds to the score for mixed questions
            total_incorrect_answers += incorrect_answers_mixed # Adds to total number of incorrect numbers
        mixed_question_avg = total_time2 / len(numbers_list3) # Calculates the average time per question
    elif func == mixed_questions2: # This code is run when mixed questions 2 are called on
        for num_six, num_seven, num_eight in zip(numbers_list6, numbers_list7, numbers_list8): # Uses numbers from list 6, 7, 8
            clear_output(wait=True)
            time_taken3, counter_mixed2, incorrect_answers_mixed2 = mixed_questions2(num_six, num_seven, num_eight)
            total_time3 += time_taken3 # Adds together the time for each question in this question
            total_counter_mixed2 += counter_mixed2 # Adds to the score for mixed questions 2
            total_incorrect_answers += incorrect_answers_mixed2  # Adds to total number of incorrect numbers
        mixed2_question_avg = total_time3 / len(numbers_list6) # Calculates the average time per question

clear_output(wait=True)

average_time = (int(total_time) + int(total_time2) + int(total_time3)) / 15 # Calculates thr average time per question
counter = total_counter_add + total_counter_mixed + total_counter_mixed2 # Calculates the total score

# These display to  the user their score anf time
display(HTML(f"<h1>The average time to answer each question is: {average_time:.2f}"))
display(HTML(f"<h1>Your final score is: {counter}/30<h/1>"))
time.sleep(4)
clear_output(wait=True)

# This dictionary stores the users answers, so they can be uploaded to google sheets
data_dict = {
    'id': user_id,
    'age': ans1,
    'gender': ans2,
    'substance': ans3,
    'addition time': total_time,
    'addition score': total_counter_add,
    'addition incorrect': incorrect_answers_add,
    'mixed time': total_time2,
    'mixed score': total_counter_mixed,
    'mixed incorrect': incorrect_answers_mixed,
    'mixed 2 time': total_time3,
    'mixed 2 score': total_counter_mixed2,
    'mixed 2 incorrect': incorrect_answers_mixed2,
    'average time': average_time,
    'counter': counter,
    'total incorrect': total_incorrect_answers
}

btn1 = widgets.Button(description="Yes")
btn2 = widgets.Button(description="No")

btn1.on_click(register_event)
btn2.on_click(register_event)

# To give the user the option of not uploading their scores and answers, there is a button and the score is only uploaded if they click yes
data_consent_info = """
Please read: <br>
We wish to store you response data to an anonymised public data repository. Your
data will be used for educational teaching purposes practising data analysis and
visualisation. <br>
Please press yes if you consent to your data being uploaded <br>
You have 15 seconds to answer"""
display(HTML(f"<h1>{data_consent_info}</h1>"))

panel = widgets.HBox([btn1, btn2])
display(panel)

result = wait_for_event(timeout=15)
clear_output()

if result['description'] == "Yes":
    display(HTML("<h1>Your answers will be uploaded to Google Forms<?h1>"))
    display(HTML("<h1>Thank you for participating in this test! Your data has been uploaded<?h1>"))
    display(HTML("<h1>PLease contact a.fedorec@ucl.ac.uk</h1>"))
    display(HTML("<h1>If you have any questions or concerns regarding the stored results</h1>"))
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSfvTT-Bj4nQfALiUdBo-zcydTC9uHH3XecY4ReK4_tBJf9HQQ/viewform?usp=sf_link'
    send_to_google_form(data_dict, form_url)
else:
    display(HTML("<h1>No problem we hope you enjoyed the test</h1>"))